Notes on MLPP's chap 12: Variational Inference:
- 

Notes
- I'm changing the notation on $P$ here for a markov network to be $P_{\Phi}$

Notes from PGM on Mean Field
- using a product of marginals makes inference easier. We need only consider the marginals involving our query variables.

What needs to be mentioned?

- Parameters can be viewed as hidden variables..
- I haven't even mentioned hidden variables yet!
- How does conditioning on $\mathbf{x}$ relate to conditioning on $D$ (think of many Gibbs table and you average their answer)

PGM: Sec 11.5.1 The Mean Field Approximation

- The algo performs message passing where the messages are distributions over single cariables
- It finds the distribution that is closest to $P_\Phi$ and is only a factor of marginals.
- It's computational attractive because we only need to consider the marginals of the variables involved in our query.
- This is the 'energy functional': $\mathbb{H}[\textrm{Q}] - \mathbb{H}[\textrm{Q},\tilde{P}_\Phi]$. We are maximizes it when we reduce KL divergence. It approaches $\log Z$.
- Consider each term. Consider trying to make $ - \mathbb{H}[\textrm{Q},\tilde{P}_\Phi]$ large. This is just:

$$
\begin{align}
 - \mathbb{H}[\textrm{Q},\tilde{P}_\Phi] & = \sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \log \tilde{P}_\Phi(\mathbf{x}) \\
 & = \sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \big[\log \prod_{\phi_i\in \Phi} \phi_i(\mathbf{D}_i) \big]\\
  & = \sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \big[ \sum_{\phi_i\in \Phi} \log \phi_i(\mathbf{D}_i) \big]\\
  & = \sum_{\phi_i\in \Phi} \sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x})  \log \phi_i(\mathbf{D}_i) \\
  & = \sum_{\phi_i\in \Phi} \mathbb{E}_{\mathbf{D}_i\sim \textrm{Q}}\big[\log \phi_i(\mathbf{D}_i) \big]
\end{align}
$$

- $\mathbb{H}[\textrm{Q}]$ also decomposes. It decomposes into a sum of the marginal entropies.

# What is Variance Inference?

(This answer is the 3rd answer in a 7 part series on Probabilistic Graphical Models)

We've defined inference as:

"The task of using a given graphical model of a system, complete with fitted parameters, to answer questions regarding that system."

As you may know by this point, inference is the task of using a model of a system to calculate answers regarding that system. 'Variational' inference refers to a specific technique to do so *approximately* - a necessary consession when the exact alternative is hopelessly expensive. We'll discover that it relies on recasting inference as an optimization problem, which comes with a few benefits:

1. Many optimization techniques are suddenly at our disposal
2. We have additional insight into the nature of our problem.
3. A by-product of our calculations is very useful for model selection.

But first, I'll recap all that is necessary to understand this answer.

### A refresher

In the first answer [link], we discovered why Probabilistic Graphical Models (PGMs) are useful tools for studying complex system. We defined a complex system as a set of $n$ random variables (which we'll call $\mathcal{X}$, so $\mathcal{X}=\{X_1,\cdots,X_n\}$ for example) with a relationship we'd like to understand. We are given some, at least partial, joint observations of $\mathcal{X}$. Performing *inference* means using a given PGM, complete with parameters, to answer:

1. **Probability Queries**: Compute the probabilities $P(\mathbf{Y}|\mathbf{X}=\mathbf{x})$. What is the distribution of the RV's of $\mathbf{Y}$ given we have some observation of the RVs of $\mathbf{X}$?
2. **MAP Queries**: Determine $\textrm{argmax}_\mathbf{y}P(\mathbf{Y}=\mathbf{y}|\mathbf{X}=\mathbf{x})$ when given $\mathbf{x}$. That is, determine the most likely values of random variables (RVs) given an assignment of other RVs.

(Where $\mathbf{X}$ and $\mathbf{Y}$ are two arbitrary non-overlapping subsets of $\mathcal{X}$ and $\mathbf{x}$ is an observed assignment of $\mathbf{X}$. If this notation is unfamiliar, see the 'Preamble Notes' section from the last answer).

We discovered that even if we had the joint distribution of $\mathcal{X}$ (called $P$), we could only answer these questions if we had certain simplifications on $P$. Those simplifications were **conditional independence (CI) statements**. If such statements are true for $P$ (stated as '$P$ satisfies these CI statements'), it simplifies calculations on $P$.

The big innovation of PGMs is that we may *represent* sets of CI statement with a graph of nodes and edges, and therefore, represent all $P$'s which satisfy them. The two dominate types of PGMs are the Bayesian Network (BN) and the Markov Network (MN). A BN is associated with a graph with *directed* edges (a DAG) and uses the Chain Rule, along with Conditional Probability Tables ('CPTs' or 'CPDs'), to determine probabilities. A choice of CPDs makes the BN recreate the probabilities of a specific $P$. Similarly, a MN is associated with an *undirected* graph, which may represent a different class of CI statements. Instead of the Chain Rule, a MN uses functions called *factors* to calculate probabilities. I called this the 'Gibbs Rule'.

Since it'll be important, let's review the Gibbs Rule in more detail. Let's say we have an MN graph (call it $\mathcal{H}$) with nodes for each RV of $\mathcal{X}$. The set of nodes for each complete subgraph are given by $\{\mathbf{D}_i\}_{i=1}^m$ and for each of these, we have an associated factor, the set of which can be represented as $\Phi=\{\phi_i(\cdots)\}_{i=1}^m$. A factor is just a function that maps from some assignment to a positive (nonnegative?) number. With that, the Gibbs Rule tells us to calculate probabilities as:

$$
P_\Phi(X_1,\cdots,X_n) = \frac{1}{Z} \prod_{\phi_i\in \Phi} \phi_i(\mathbf{D}_i)
$$

where

$$
Z = \sum_{\mathbf{x}\in Val(\mathcal{X})} \prod_{\phi_i\in \Phi} \phi_i(\mathbf{D}_i)
$$

For a given assignment, we say that $\prod_{\phi_i\in \Phi} \phi_i(\mathbf{D}_i)$ is the 'unnormalized probability', often abbreviated as $\tilde{P}_\Phi(\cdots)$. To crystallize this idea, it's helpful to imagine the 'Gibbs Table', which lists unnormalized probabilities for all assignments. In the second answer [link], we pictured an example where $\mathcal{X}=\{C,D,I,G,S\}$ as:

[need to fix make the $P \rightarrow P_\Phi$ change in this image]

![Title](GIbbsTable_labeled.png)

Speaking of the the second answer [link] - it's not a prerequisite for this, but it's necessary to steal two other ideas from it.

The first is that we can always recreate the probabilities produced by a BN's Chain Rule with an another invented MN and its Gibbs Rule. Essentially, we define factors that reproduce a BN's CPDs to do. This equivalence allows us to reason solely in terms of the Gibbs Rule, while assured that whatever we discover will also hold for BNs.

The second is that inference ultimately reduces to two tasks. For performing probability queries, it reduces to summing up unnormalized probabilities of the Gibbs Table when we filter to rows that have some assignment of some subsets of $\mathcal{X}$. For MAP queries, it comes down to finding max unnormalized probabilities after this filtering[1].

### The big problem

The issue is that the Gibbs Table may have exponentially many rows (assignments) and we can't sum effeciently across them. So that means we can't calculate $Z$ for starters. Exact inference algorithms help, but their 'exact' constraint puts significant limits on their speed. There are some graphs for which exact inference just isn't an option.

So we need to be clever.

### The big idea

The idea behind Variational Inference is to consider a space of 'simple' probability distributions and pick one that closely approximates $P_\Phi$. With one selected, we can perform inference on this distribution rather than $P_\Phi$. The 'simple' constraint ensures such inference is feasible. To do this, we need to determine this space, have a method to search it and have a measure that tells us how well one distribution approximates another.

Let's say that with notation. Let $\mathcal{Q}$ be a space of distribution and let $\textrm{Q}$ refer to any distribution from $\mathcal{Q}$. We want to answer the optimization of:

$$
\textrm{Q}^* = \textrm{argmin}_{\textrm{Q} \in \mathcal{Q}} D(\textrm{Q},P_\Phi)
$$

where $D(\textrm{Q},P_\Phi)$ is some yet-to-be-defined measure that tells us how similar $\textrm{Q}$ and $P_\Phi$ are. With this, we can perform inference on $\textrm{Q}^*$ in place of $P_\Phi$.

The first question to ask is: what is our measure of proximity, $D(\cdot,\cdot)$?

### A measure between two distributions

Recognize how strange this task is. $\textrm{Q}$ and $P_\Phi$ are *distributions* that map assignments to probabilities. How can we determine whether one approximates another well?

Luckily, our information theory pals threw us a lifesaver. That lifesaver is KL-divergence [link], which accepts two probability distributions and returns a non-negative number. If that number is large, then the distributions are very different. If it zero, the distributions are identical. We write KL-divergence as $\mathbb{KL}(\textrm{Q}||P_\Phi)$. However, there is one ugly property of KL-divergence - it's not symmetric. In general, $\mathbb{KL}(Q||P_\Phi) \neq \mathbb{KL}(P_\Phi||Q)$.

That said, due to $P_\Phi$'s unweildiness, we are forced to use $\mathbb{KL}(\textrm{Q}||P_\Phi)$, which is defined as:

$$
\begin{align}
\mathbb{KL}(\textrm{Q}||P_\Phi) = \sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \log \frac{\textrm{Q}(\mathbf{x})}{P_\Phi(\mathbf{x})}\\
\end{align}
$$

If we sub in $\frac{1}{Z}\tilde{P}_\Phi(\mathbf{x})$ for $P_\Phi(\mathbf{x})$, re-arrange things a bit and label what we recognize, we get this neat fella:

$$
\mathbb{KL}(\textrm{Q}||P_\Phi) = \underbrace{\big(-\sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \log \tilde{P}_\Phi(\mathbf{x})\big)}_{\textrm{'Cross Entropy' of }\textrm{Q and } \tilde{P}_\Phi,\space \mathbb{H}[\textrm{Q},\tilde{P}_\Phi]} - \underbrace{\big(-\sum_{\mathbf{x} \in Val(\mathbf{X})} \textrm{Q}(\mathbf{x}) \log \textrm{Q}(\mathbf{x})\big)}_{\textrm{Entropy of Q},\space \mathbb{H}[\textrm{Q}]} +\log Z
$$

This new notation is to guide how we should think about the components of KL-divergence:

1. $\mathbb{H}[\textrm{Q},\tilde{P}_\Phi]$: This is called the Cross Entropy of $\textrm{Q}$ and $\tilde{P}_\Phi$. It's a measure of *dis*agreement. When this number is high, that means $\textrm{Q}$ puts high probability on $\mathbf{x}$'s that $\tilde{P}_\Phi$ believes are rare.[2]
2. $\mathbb{H}[\textrm{Q}]$: The entropy of $\textrm{Q}$ - a number that measures uncertainty. High entropy implies $\textrm{Q}$ has its probability spread out very thinly and evenly. Low entropy implies it has concentrated probabilities on a small number of $\mathbf{x}$'s. I wrote more about this subject here [link].
3. $\log Z$: This is the log normalizer of $P_\Phi$. The choice of $\textrm{Q}$ can't impact this, so in the context of the optimization, we ignore it.

So when we look at:

$$
\mathbb{KL}(\textrm{Q}||P_\Phi) = \mathbb{H}[\textrm{Q},\tilde{P}_\Phi] - \mathbb{H}[\textrm{Q}] +\log Z
$$

We should think: "Ok, a $\textrm{Q}$ that does a good job (minimizes $\mathbb{KL}(\textrm{Q}||P_\Phi)$) will put high probabilities on $\mathbf{x}$'s that $\tilde{P}_\Phi$ puts high weight on (low $\mathbb{H}[\textrm{Q},\tilde{P}_\Phi]$) *and* it'll have it's probabilities spread out thinly (high $\mathbb{H}[\textrm{Q}]$)." The latter term, $\mathbb{H}[\textrm{Q}]$, might be surprising. (Explain the latter term)

That sounds nice, but this behavior isn't why this result is such a big deal. That comes down to two other reasons.

The first is that we *don't* need to calculate $P_\Phi(\mathbf{x})$ ever, which is intractable due to $Z$. We can optimize to the unnormalized version and, theoretically, we're assured to get the same result as though we did calculate $P_\Phi(\mathbf{x})$.

The second helps us with that intractable $Z$. Let's say we pull off a perfect optimization and found a $\textrm{Q}^*$ that obtained a KL-divergence of zero[3]. Well, that would imply:

$$
\mathbb{H}[\textrm{Q}^*] - \mathbb{H}[\textrm{Q}^*,\tilde{P}_\Phi] = \log Z
$$

Since we obtained $\textrm{Q}^*$ by optimizing the terms of the left hand side, we know their values and hence, we now know $Z$. This is huge!

Now, we won't actually get a $\textrm{Q}^*$ that obtains a KL-divergence of 0. But, the better we optimize, the closer this quantity will approach $\log Z$. So it's an *approximation*:

$$
\begin{align}
\mathbb{H}[\textrm{Q}] - \mathbb{H}[\textrm{Q},\tilde{P}_\Phi] & \rightarrow \log Z \\
\textrm{ as }\mathbb{KL}(\textrm{Q}||P_\Phi) & \rightarrow 0
\end{align}
$$

It's also a lower bound, so $\mathbb{H}[\textrm{Q}] - \mathbb{H}[\textrm{Q},\tilde{P}_\Phi] \leq \log Z$ the whole time.

This is useful because as we are doing inference, we get an approximation to $Z$ as a by product. (Explain why this is useful).


### The Mean Field method

We still have two more questions to answer: what is our space, $\mathcal{Q}$, and how do we search it? Resolving these will land us on a specific Varitional Inference algorithm. Our answers here will land us on the Mean Field method.

So, what is our space, $\mathcal{Q}$? How about: all distributions that can be expressed as products of univariate distributions. 'Univariate' means each accept a singular assignment of one variable and return a probability. So if there are $m$ variables in $\mathbf{X}$, then we could expression this as:

$$
\{ \textrm{Q} \in \mathcal{Q} \textrm{ if Q}=\prod_{i=1}^m q_i(X_i)\textrm{ for some univariate }q_i(\cdot)'s\}
$$

Great, now how do we search this space? Well our choice of $D(\cdot,\cdot)$ and $\mathcal{Q}$ leads us to a particular simple search strategy. Let's write out our optimization problem with these specifics filled in. Since a $\textrm{Q}$ is defined by a set of $q_i(\cdot)$'s and we are using KL-divergence, our task is to find:

$$
\textrm{argmin}_{\{q_i(\cdot)\}'s} \mathbb{KL}\big[\prod_i q_i(\cdot)\big|\big|P_\Phi\big]
$$

To do this, we have an algorithm which updates each $q_i(\cdot)$'s iteratively. This falls out of using the Gibbs Rule for $P_\Phi$ and minimizing the KL-divergence with respect to one $q_j(\cdot)$ given all the *other* $q_i(\cdot)$'s. 

The algorithm starts by initializing all $q_i(\cdot)$'s to the uniform distribution. Then, we pick one (say $q_j(\cdot)$) to update by setting it to:

$$
\begin{align}
q_j(X_j) & = \frac{1}{Z_j}\exp\big(\sum_{i:X_j \in \mathbf{D}_i} \mathbb{E}_{\mathbf{D}_i \setminus X_j \sim \textrm{Q}}\big[\log \phi_i(\mathbf{D}_i)\big]\big) \\
& = \frac{1}{Z_j}\exp\big(\sum_{i:X_j \in \mathbf{D}_i} \sum_{\mathbf{D}_i}\log \phi_i(\mathbf{D}_i)\big[\prod_{X_k \in \mathbf{D}_i \setminus X_j} q_k(X_k)\big] \big) \\
\end{align}
$$

[This notation needs to be improved]

[Maybe we mention the idea that this is sort-of, message passing. Each marginal depends on other marginals (message)]

The derivation of these updates is a bit complicated, but we can still make some useful observations:

1. It's remarkable that this update reduces KL-divergence to the whole of $P_\Phi$ but we need only consider factors for which $X_j$ is involved. We owe the tractability of this approach to this fact.
2. Since $X_j$ is a single variable, the function is sufficiently small that we can calculate the unnormalized probability for each assignment and then sum them up to get the normalizer $Z_j$.

After we run through these algorithms, we have a set of distributions, $\{q_i^*(X_i)\}$, that defined our approximation, $\textrm{Q}^*$, to $P_\Phi$. This factored form makes inference very easy.

### Footnotes

[1] This is only the case when $\mathbf{X}$ and $\mathbf{Y}$ make up the whole of $\mathcal{X}$.

[2] Well technically, it's not a legal Cross Entropy. Cross Entropy is defined over distributions. In this case, it's used for an unnormalized distribution $\tilde{P}_\Phi$.

[3] This would imply $\textrm{Q}$ *is* $P_\Phi$ in fact.

[4] See XYZ section of MLPP for the derivation.

### Sources

[1] The Monster

$$
\begin{align}
\mathbb{KL}(Q||P_\Phi) & = \mathbb{E}_{\mathbf{x} \sim Q}\big[\log \frac{Q(\mathbf{x})}{P_\Phi(\mathbf{x})}\big]\\
& = \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log \frac{Q(\mathbf{x})}{P_\Phi(\mathbf{x})}\\
\end{align}
$$


$$
\begin{align}
\mathbb{KL}(Q||P_\Phi) & = \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log \frac{Q(\mathbf{x})}{\frac{1}{Z}\tilde{P}_\Phi(\mathbf{x})}\\
& = \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log Q(\mathbf{x}) - \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log \frac{1}{Z}\tilde{P}_\Phi(\mathbf{x}) \\
& = \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log Q(\mathbf{x}) + \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log Z - \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log \tilde{P}_\Phi(\mathbf{x}) \\
& = \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log Q(\mathbf{x}) + \log Z - \sum_{\mathbf{x} \in Val(\mathbf{X})} Q(\mathbf{x}) \log \tilde{P}_\Phi(\mathbf{x}) \\
\end{align}
$$


To do this, we can use the following algorithm. Initialize all $q_i(\cdot)$'s to the uniform distribution. 
Then, pick one (say $q_j(\cdot)$) to update by setting it to:

$$
q_j(X_j) = \exp\big(\sum_{\mathbf{X} \setminus X_j}\prod_{i:i\neq j} q_i(X_i) \log \tilde{P}_\Phi(\mathbf{X}) \big)
$$

(It's implied that the assignment of $X_j$ is plugged into $\mathbf{X}$)

I'm not deriving it, but take it as truth that with each update, we decrease the KL-divergence[4]. We repeatedly iterative over $q_i()'s$ until they stop changing. In the end, we have a set of $q_i(\cdot)$ which together give us our $\textrm{Q}^*$.

[Explain why that update equation is remarkable]

[DO WE NEED THIS EXPLANATION? I'M PROBABLY DOING IT BECAUSE I DIDN"T UNDERSTAND HOW IT WORKS!]

Some of you might look at that update and say: "Hey! We can't sum over $\mathbf{X}$ - there are too many assignments!" Yes, you're correct. It's time for us to use the Gibbs Rule. That tells us:

$$
\tilde{P}_\Phi(X_1,\cdots,X_n) = \prod_{\phi_i\in \Phi} \phi_i(\mathbf{D}_i)
$$

Now, let's split this product into two products: one of which is the product of all factors which involve $X_j$ (call it $\psi_{X_j}(\mathbf{C}_{X_j})$) and the other is a product of everything else (call it $\psi_{\overline{X_j}}(\mathbf{C}_{\overline{X_j}})$). This is just some factor organization that gives us:

$$
\tilde{P}_\Phi(X_1,\cdots,X_n) = \psi_{X_j}(\mathbf{C}_{X_j}) \psi_{\overline{X_j}}(\mathbf{C}_{\overline{X_j}})
$$

If we plug this into our update equation, we get:

$$
\begin{align}
q_j(X_j) & = \exp\bigg(\sum_{\mathbf{X} \setminus X_j}\big(\log \psi_{X_j}(\mathbf{C}_{X_j}) + \log \psi_{\overline{X_j}}(\mathbf{C}_{\overline{X_j}})\big)\prod_{i:i\neq j} q_i(X_i)  \bigg)\\
& = \exp\bigg(\sum_{\mathbf{X} \setminus X_j}\log \psi_{X_j}(\mathbf{C}_{X_j})\prod_{i:i\neq j} q_i(X_i) + \underbrace{\sum_{\mathbf{X} \setminus X_j}\log \psi_{\overline{X_j}}(\mathbf{C}_{\overline{X_j}})\prod_{i:i\neq j}q_i(X_i) }_{\text{Doesn't involve }X_j \text{ - it's a constant!}}\bigg)\\
& = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus X_j}\log \psi_{X_j}(\mathbf{C}_{X_j})\prod_{i:i\neq j} q_i(X_i) \bigg)\\
\end{align}
$$

Since $q_j(X_j)$ gets multiplied by the same constant, regardless of the input, and we know these $q_j(\cdot)$ is a normalized distribution, then *this constant doesn't matter*. We can ignore it! Ahh!

But we are still summing over something nearly as big as $\mathbf{X}$, but we can make progress:

$$
\begin{align}
q_j(X_j) & = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus X_j}\log \psi_{X_j}(\mathbf{C}_{X_j})\prod_{i:i\neq j} q_i(X_i) \bigg)\\
& = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus \mathbf{C}_{X_j}} \sum_{\mathbf{C}_{X_j}}\log \psi_{X_j}(\mathbf{C}_{X_j})\big[\prod_{X_k \in \mathbf{X} \setminus \mathbf{C}_{X_j}}q_k(X_k)\big]\big[\prod_{X_i \in \mathbf{C}_{X_j}} q_i(X_i)\big] \bigg)\\
& = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus \mathbf{C}_{X_j}}\big[\prod_{X_k \in \mathbf{X} \setminus \mathbf{C}_{X_j}}q_k(X_k)\big] \sum_{\mathbf{C}_{X_j}}\log \psi_{X_j}(\mathbf{C}_{X_j})\big[\prod_{X_i \in \mathbf{C}_{X_j}} q_i(X_i)\big] \bigg)\\
\end{align}
$$

$$
\begin{align}
q_j(X_j) & = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus X_j}\prod_{i:i\neq j} q_i(X_i) \log \psi_{X_j}(\mathbf{C}_{X_j})\bigg)\\
& = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus X_j}\big[\prod_{X_k \in \mathbf{X} \setminus \mathbf{C}_{X_j}}q_k(X_k)\big]\big[\prod_{X_i \in \mathbf{C}_{X_j}} q_i(X_i) \log \psi_{X_j}(\mathbf{C}_{X_j})\big]\bigg)\\
& = \text{const}\cdot\exp\bigg(\sum_{\mathbf{X} \setminus \mathbf{C}_{X_j}} \sum_{\mathbf{C}_{X_j}}\big[\prod_{X_k \in \mathbf{X} \setminus \mathbf{C}_{X_j}}q_k(X_k)\big]\big[\prod_{X_i \in \mathbf{C}_{X_j}} q_i(X_i) \log \psi_{X_j}(\mathbf{C}_{X_j})\big]\bigg)\\
\end{align}
$$